In [4]:
import cv2
import numpy as np
from pathlib import Path
import random
import os

In [5]:
# Configuration for Random Cropping (Cell 2)

# Input/Output directories
RANDOM_CROP_INPUT = r"C:\Users\thaim\Videos\AI_LEDS\no_detector_pictures"
RANDOM_CROP_OUTPUT = r"C:\Users\thaim\Videos\AI_LEDS\cropped_no_detector_pictures"

# Crop sizes (in pixels)
CROP_SIZES = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200]

# Number of crops to generate per size
CROPS_PER_SIZE = 10

In [ ]:
def extract_random_crops(input_dir, output_dir, crop_sizes, crops_per_size):
    """Extract random crops from each image with varying sizes"""
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    
    if not input_path.exists():
        print(f"Error: Input directory not found: {input_dir}")
        return
    
    output_path.mkdir(parents=True, exist_ok=True)
    
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    images = []
    for ext in image_extensions:
        images.extend(input_path.rglob(f'*{ext}'))
    
    if not images:
        print("No images found")
        return
    
    print(f"Found {len(images)} images to process")
    print(f"Will generate {len(crop_sizes)} sizes × {crops_per_size} crops = {len(crop_sizes) * crops_per_size} crops per image")
    print(f"Total crops: {len(images) * len(crop_sizes) * crops_per_size}")
    print("="*60)
    
    processed_images = 0
    total_crops = 0
    
    for img_idx, img_path in enumerate(images, 1):
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"✗ Could not read: {img_path.name}")
            continue
        
        height, width = img.shape[:2]
        
        img_folder = output_path / img_path.stem
        img_folder.mkdir(exist_ok=True)
        
        print(f"[{img_idx}/{len(images)}] Processing: {img_path.name} ({width}×{height})")
        
        for crop_size in crop_sizes:
            if crop_size > width or crop_size > height:
                print(f"  ⚠️ Skipping {crop_size}×{crop_size} (image too small)")
                continue
            
            size_folder = img_folder / f"{crop_size}x{crop_size}"
            size_folder.mkdir(exist_ok=True)
            
            used_positions = []
            crops_generated = 0
            attempts = 0
            max_attempts = crops_per_size * 100
            
            while crops_generated < crops_per_size and attempts < max_attempts:
                attempts += 1
                
                x = random.randint(0, width - crop_size)
                y = random.randint(0, height - crop_size)
                
                overlap = False
                for prev_x, prev_y, prev_size in used_positions:
                    overlap_area = max(0, min(x + crop_size, prev_x + prev_size) - max(x, prev_x)) * \
                                   max(0, min(y + crop_size, prev_y + prev_size) - max(y, prev_y))
                    crop_area = crop_size * crop_size
                    overlap_ratio = overlap_area / crop_area
                    
                    if overlap_ratio >= 0.9:
                        overlap = True
                        break
                
                if not overlap:
                    crop = img[y:y+crop_size, x:x+crop_size]
                    crop_filename = size_folder / f"crop_{crops_generated+1:03d}.jpg"
                    cv2.imwrite(str(crop_filename), crop, [cv2.IMWRITE_JPEG_QUALITY, 95])
                    
                    used_positions.append((x, y, crop_size))
                    crops_generated += 1
                    total_crops += 1
            
            if crops_generated < crops_per_size:
                print(f"  ⚠️ Only generated {crops_generated}/{crops_per_size} crops for {crop_size}×{crop_size}")
        
        processed_images += 1
        print(f"  ✓ Generated crops in: {img_folder}")
    
    print("="*60)
    print(f"SUMMARY")
    print("="*60)
    print(f"Images processed: {processed_images}/{len(images)}")
    print(f"Total crops generated: {total_crops}")
    print("="*60)

# Run extraction with configuration from Cell 2
extract_random_crops(RANDOM_CROP_INPUT, RANDOM_CROP_OUTPUT, CROP_SIZES, CROPS_PER_SIZE)

Found 17523 images to process
Will generate 10 sizes × 10 crops = 100 crops per image
Total crops: 1752300
[1/17523] Processing: frame_00000.jpg (1440×1080)
  ✓ Generated crops in: C:\Users\thaim\Videos\AI_LEDS\cropped_no_detector_pictures\frame_00000
[2/17523] Processing: frame_00001.jpg (1440×1080)
  ✓ Generated crops in: C:\Users\thaim\Videos\AI_LEDS\cropped_no_detector_pictures\frame_00001
[3/17523] Processing: frame_00002.jpg (1440×1080)
  ✓ Generated crops in: C:\Users\thaim\Videos\AI_LEDS\cropped_no_detector_pictures\frame_00002
[4/17523] Processing: frame_00003.jpg (1440×1080)
  ✓ Generated crops in: C:\Users\thaim\Videos\AI_LEDS\cropped_no_detector_pictures\frame_00003
[5/17523] Processing: frame_00004.jpg (1440×1080)
  ✓ Generated crops in: C:\Users\thaim\Videos\AI_LEDS\cropped_no_detector_pictures\frame_00004
[6/17523] Processing: frame_00005.jpg (1440×1080)
  ✓ Generated crops in: C:\Users\thaim\Videos\AI_LEDS\cropped_no_detector_pictures\frame_00005
[7/17523] Processing: f

In [9]:
class ROISelector:
    def __init__(self):
        self.roi = None
        self.drawing = False
        self.start_point = None
        self.image = None
        self.clone = None
    
    def select_roi(self, image_path):
        """Let user select ROI from an image"""
        img = cv2.imread(str(image_path))
        if img is None:
            print(f"Error: Cannot read image {image_path}")
            return None
        
        self.image = img.copy()
        self.clone = img.copy()
        
        cv2.namedWindow("Select ROI - Press ENTER when done, 'r' to reset")
        cv2.setMouseCallback("Select ROI - Press ENTER when done, 'r' to reset", self.mouse_callback)
        
        print("\nDraw a rectangle for the ROI:")
        print("- Click and drag to draw rectangle")
        print("- Press 'r' to reset")
        print("- Press ENTER when done")
        print("- Press ESC to skip this folder\n")
        
        while True:
            cv2.imshow("Select ROI - Press ENTER when done, 'r' to reset", self.image)
            key = cv2.waitKey(1) & 0xFF
            
            if key == 13:
                if self.roi is not None:
                    break
            elif key == ord('r'):
                self.image = self.clone.copy()
                self.roi = None
            elif key == 27:
                cv2.destroyAllWindows()
                return None
        
        cv2.destroyAllWindows()
        return self.roi
    
    def mouse_callback(self, event, x, y, flags, param):
        """Handle mouse events for ROI selection"""
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.start_point = (x, y)
        
        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing:
                self.image = self.clone.copy()
                cv2.rectangle(self.image, self.start_point, (x, y), (0, 255, 0), 2)
        
        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False
            cv2.rectangle(self.image, self.start_point, (x, y), (0, 255, 0), 2)
            x1, y1 = self.start_point
            x2, y2 = x, y
            self.roi = (min(x1, x2), min(y1, y2), abs(x2 - x1), abs(y2 - y1))

def crop_images_with_roi(input_dir, output_dir):
    """Crop all images in subfolders using one ROI per subfolder"""
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    
    if not input_path.exists():
        print(f"Error: Input directory not found: {input_dir}")
        return
    
    output_path.mkdir(parents=True, exist_ok=True)
    
    subfolders = [f for f in input_path.iterdir() if f.is_dir()]
    
    if not subfolders:
        print("No subfolders found")
        return
    
    print(f"Found {len(subfolders)} subfolders to process")
    print("="*60)
    
    processed_folders = 0
    skipped_folders = 0
    total_images = 0
    
    for folder_idx, subfolder in enumerate(subfolders, 1):
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
        images = []
        for ext in image_extensions:
            images.extend(subfolder.glob(f'*{ext}'))
        
        if not images:
            print(f"[{folder_idx}/{len(subfolders)}] Skipping {subfolder.name} (no images)")
            skipped_folders += 1
            continue
        
        print(f"\n[{folder_idx}/{len(subfolders)}] Folder: {subfolder.name} ({len(images)} images)")
        
        first_image = images[0]
        roi_selector = ROISelector()
        roi = roi_selector.select_roi(first_image)
        
        if roi is None:
            print(f"  Skipped by user")
            skipped_folders += 1
            continue
        
        x, y, w, h = roi
        print(f"  ROI selected: x={x}, y={y}, w={w}, h={h}")
        
        output_subfolder_name = f"{subfolder.name}_{w}x{h}"
        output_subfolder = output_path / output_subfolder_name
        output_subfolder.mkdir(exist_ok=True)
        
        for img_path in images:
            img = cv2.imread(str(img_path))
            if img is None:
                print(f"  ✗ Could not read: {img_path.name}")
                continue
            
            cropped = img[y:y+h, x:x+w]
            
            img_stem = img_path.stem
            img_ext = img_path.suffix
            output_img_name = f"{img_stem}_{w}x{h}{img_ext}"
            output_img_path = output_subfolder / output_img_name
            
            cv2.imwrite(str(output_img_path), cropped, [cv2.IMWRITE_JPEG_QUALITY, 95])
            total_images += 1
        
        print(f"  ✓ Cropped {len(images)} images → {output_subfolder}")
        processed_folders += 1
    
    print("\n" + "="*60)
    print(f"SUMMARY")
    print("="*60)
    print(f"Folders processed: {processed_folders}/{len(subfolders)}")
    print(f"Folders skipped: {skipped_folders}")
    print(f"Total images cropped: {total_images}")
    print("="*60)

# Configuration
INPUT_DIR = r"C:\Users\thaim\Videos\AI_LEDS\detector_pictures"
OUTPUT_DIR = r"C:\Users\thaim\Videos\AI_LEDS\cropped_detector_pictures"

# Run ROI cropping
crop_images_with_roi(INPUT_DIR, OUTPUT_DIR)

Found 43 subfolders to process

[1/43] Folder: 001_40_mtr_rka_la_shchvr_3_ldym_shmalh_1 (708 images)

Draw a rectangle for the ROI:
- Click and drag to draw rectangle
- Press 'r' to reset
- Press ENTER when done
- Press ESC to skip this folder

  ROI selected: x=457, y=585, w=39, h=65
  ✓ Cropped 708 images → C:\Users\thaim\Videos\AI_LEDS\cropped_detector_pictures\001_40_mtr_rka_la_shchvr_3_ldym_shmalh_1_39x65

[2/43] Folder: 002_rka_shchvr_lglay_40_mtr_shmalh_3_ldym_1 (908 images)

Draw a rectangle for the ROI:
- Click and drag to draw rectangle
- Press 'r' to reset
- Press ENTER when done
- Press ESC to skip this folder

  ROI selected: x=451, y=588, w=33, h=58
  ✓ Cropped 908 images → C:\Users\thaim\Videos\AI_LEDS\cropped_detector_pictures\002_rka_shchvr_lglay_40_mtr_shmalh_3_ldym_1_33x58

[3/43] Folder: 005_30_rgyl_ayn_shmsh_ldym_mkdymh_rka_shkvf (1917 images)

Draw a rectangle for the ROI:
- Click and drag to draw rectangle
- Press 'r' to reset
- Press ENTER when done
- Press ESC 